In [41]:
import yfinance as yf
import pandas as pd
import numpy as np
from pymongo import MongoClient
import datetime
import matplotlib.pyplot as plt
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
from talib import abstract
import math
## This is to Convert a string representation of list into list 
import ast
from functools import partial
import os
from ta.momentum import rsi
from ta.utils import dropna
# import pandas_ta as pta
from collections import defaultdict


In [9]:
def Supertrend(df, atr_period, multiplier):
    
    high = df['High']
    low = df['Low']
    close = df['Close']
    
    # calculate ATR
    price_diffs = [high - low, 
                   high - close.shift(), 
                   close.shift() - low]
    true_range = pd.concat(price_diffs, axis=1)
    true_range = true_range.abs().max(axis=1)
    # default ATR calculation in supertrend indicator
    atr = true_range.ewm(alpha=1/atr_period,min_periods=atr_period).mean() 
    # df['atr'] = df['tr'].rolling(atr_period).mean()
    
    # HL2 is simply the average of high and low prices
    hl2 = (high + low) / 2
    # upperband and lowerband calculation
    # notice that final bands are set to be equal to the respective bands
    final_upperband = upperband = hl2 + (multiplier * atr)
    final_lowerband = lowerband = hl2 - (multiplier * atr)
    
    # initialize Supertrend column to True
    supertrend = [True] * len(df)
    
    for i in range(1, len(df.index)):
        curr, prev = i, i-1
        
        # if current close price crosses above upperband
        if close[curr] > final_upperband[prev]:
            supertrend[curr] = True
        # if current close price crosses below lowerband
        elif close[curr] < final_lowerband[prev]:
            supertrend[curr] = False
        # else, the trend continues
        else:
            supertrend[curr] = supertrend[prev]
            
            # adjustment to the final bands
            if supertrend[curr] == True and final_lowerband[curr] < final_lowerband[prev]:
                final_lowerband[curr] = final_lowerband[prev]
            if supertrend[curr] == False and final_upperband[curr] > final_upperband[prev]:
                final_upperband[curr] = final_upperband[prev]

        # to remove bands according to the trend direction
        if supertrend[curr] == True:
            final_upperband[curr] = np.nan
        else:
            final_lowerband[curr] = np.nan
    
    return pd.DataFrame({
        'Supertrend': supertrend,
        'Final Lowerband': final_lowerband,
        'Final Upperband': final_upperband
    }, index=df.index)

In [17]:
def resampler(df,time):
    newdf=pd.DataFrame()
    newdf['Open']=df.Open.resample(time).first()
    newdf['Close']=df.Close.resample(time).last()
    newdf['High']=df.High.resample(time).max()
    newdf['Low']=df.Low.resample(time).min()
    newdf['Tick_Volume']=df['Tick Volume'].resample(time).sum()
    newdf['Volume']=df.Volume.resample(time).sum()
    newdf.dropna(axis=0,inplace=True)
    
    return newdf

In [1]:
files=['importdatafiles/01-07--OCT-2021-WEEKLY-expiry_data__VEGE_NF_AND_BNF_Options.csv',]

In [2]:
for i in files:
    print(f'BackTesting from history file: {i}')
    df=pd.read_csv(i)
    df['Date/Time']=pd.to_datetime(df['Date/Time'].astype(str), format='%d-%m-%Y %H:%M:%S')
    df.rename(columns={'Date/Time':'Date','Volume':'Tick Volume','Open Interest':'Volume'},inplace=True)

BackTesting from history file: importdatafiles/01-07--OCT-2021-WEEKLY-expiry_data__VEGE_NF_AND_BNF_Options.csv


In [ ]:
def predf(datadf):
    datadf.set_index('Date',inplace=True)
    ###Base Time Frame########
    datadf=resampler(datadf,'5min')
    supertrendBTF=Supertrend(datadf,10,1)
    RSIBTF=rsi(datadf.Close,window=8,fillna=True)
    RSIMABTF=datadf['rsi'].rolling(13).mean()
    
    
    # datadf=datadf.join(supertrend)
    ###High Time Frame#########
    datadf=resampler(datadf,'10min')
    supertrendHTF=Supertrend(datadf,10,1)
    RSIHTF=rsi(datadf.Close,window=8,fillna=True)
    RSIMAHTF=datadf['rsi'].rolling(13).mean()
    
    
    datadf['rsi']=rsi(datadf.Close,window=8,fillna=True)
    datadf['rsima']=datadf['rsi'].rolling(13).mean()
    return datadf

In [4]:
df=df.loc[df.Ticker=='NIFTY']

In [48]:
tdf=df.copy()

In [58]:
datadf=df.copy()
datadf.set_index('Date',inplace=True)
datadf=resampler(datadf,'5min')
supertrendBTF=Supertrend(datadf,10,1)
supertrendBTF['rsi']=rsi(datadf.Close,window=8,fillna=True)
supertrendBTF['rsima']=supertrendBTF['rsi'].rolling(13).mean()
supertrendBTF.reset_index(inplace=True)

In [59]:
supertrendBTF.head()

,Date,Supertrend,Final Lowerband,Final Upperband,rsi,rsima
0,2021-10-01 09:15:00,True,NaN,NaN,100.000000,NaN
1,2021-10-01 09:20:00,True,NaN,NaN,100.000000,NaN
2,2021-10-01 09:25:00,True,NaN,NaN,100.000000,NaN
3,2021-10-01 09:30:00,True,NaN,NaN,100.000000,NaN
4,2021-10-01 09:35:00,True,NaN,NaN,71.699073,NaN


In [57]:
datadf=df.copy()
datadf.set_index('Date',inplace=True)
datadf=resampler(datadf,'10min')
supertrendHTF=Supertrend(datadf,10,1)
supertrendHTF['rsi']=rsi(datadf.Close,window=8,fillna=True)
supertrendHTF['rsima']=supertrendHTF['rsi'].rolling(13).mean()
supertrendHTF.reset_index(inplace=True)

In [60]:
supertrendHTF.head()

,Date,Supertrend,Final Lowerband,Final Upperband,rsi,rsima
0,2021-10-01 09:10:00,True,NaN,NaN,100.000000,NaN
1,2021-10-01 09:20:00,True,NaN,NaN,100.000000,NaN
2,2021-10-01 09:30:00,True,NaN,NaN,100.000000,NaN
3,2021-10-01 09:40:00,True,NaN,NaN,50.499002,NaN
4,2021-10-01 09:50:00,True,NaN,NaN,39.727608,NaN


In [68]:
st=pd.merge(supertrendBTF,supertrendHTF,on='Date',how='left')
st=st.replace(to_replace=np.nan,method='ffill')
st=st.set_index('Date')

In [70]:
datadf=df.copy()
datadf.set_index('Date',inplace=True)
datadf=resampler(datadf,'5min')
datadf=datadf.join(st)

In [71]:
datadf

,Open,Close,High,Low,Tick_Volume,Volume,Supertrend_x,Final Lowerband_x,Final Upperband_x,rsi_x,rsima_x,Supertrend_y,Final Lowerband_y,Final Upperband_y,rsi_y,rsima_y
Date,,,,,,,,,,,,,,,,
2021-10-01 09:15:00,17531.9,17487.8,17531.9,17478.3,5,0,True,NaN,NaN,100.000000,NaN,NaN,NaN,NaN,NaN,NaN
2021-10-01 09:20:00,17487.6,17502.5,17502.5,17459.3,5,0,True,NaN,NaN,100.000000,NaN,True,NaN,NaN,100.000000,NaN
2021-10-01 09:25:00,17501.8,17507.0,17512.4,17487.5,5,0,True,NaN,NaN,100.000000,NaN,True,NaN,NaN,100.000000,NaN
2021-10-01 09:30:00,17507.6,17528.0,17529.0,17497.3,5,0,True,NaN,NaN,100.000000,NaN,True,NaN,NaN,100.000000,NaN
2021-10-01 09:35:00,17526.3,17515.5,17526.3,17501.0,5,0,True,NaN,NaN,71.699073,NaN,True,NaN,NaN,100.000000,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2021-10-07 15:10:00,17796.0,17790.3,17796.8,17789.0,5,0,False,17834.541133,17807.405104,28.392946,35.944759,False,17827.410405,17812.033134,28.578844,47.058839
2021-10-07 15:15:00,17790.8,17788.8,17791.3,17785.8,5,0,False,17834.541133,17802.154593,27.553361,33.586705,False,17827.410405,17812.033134,28.578844,47.058839
2021-10-07 15:20:00,17788.6,17788.4,17789.5,17780.5,5,0,False,17834.541133,17798.144134,27.307271,32.506241,False,17827.410405,17811.699821,36.716021,44.586667


In [25]:
supertrend.reset_index(inplace=True)

In [33]:
tdf1=resampler(tdf,'3min')

In [34]:
supertrend1=Supertrend(tdf1,10,2.0)

In [35]:
supertrend1.reset_index(inplace=True)

In [38]:
st=pd.merge(supertrend,supertrend1,on='Date',how='left')

In [40]:
st.replace(to_replace=np.nan,method='ffill')

,Date,Supertrend_x,Final Lowerband_x,Final Upperband_x,Supertrend_y,Final Lowerband_y,Final Upperband_y
0,2021-10-01 09:15:00,True,NaN,NaN,True,NaN,NaN
1,2021-10-01 09:16:00,True,NaN,NaN,True,NaN,NaN
2,2021-10-01 09:17:00,True,NaN,NaN,True,NaN,NaN
3,2021-10-01 09:18:00,True,NaN,NaN,True,NaN,NaN
4,2021-10-01 09:19:00,True,NaN,NaN,True,NaN,NaN
...,...,...,...,...,...,...,...
1875,2021-10-07 15:26:00,True,17782.941906,17793.393582,False,17831.517425,17804.041163
1876,2021-10-07 15:27:00,True,17786.247716,17793.393582,False,17831.517425,17804.041163
1877,2021-10-07 15:28:00,True,17787.897944,17793.393582,False,17831.517425,17804.041163
1878,2021-10-07 15:29:00,True,17789.343150,17793.393582,False,17831.517425,17804.041163


In [24]:
df=df.join(supertrend)

In [25]:
df['rsi']=rsi(df.Close,window=8,fillna=True)

In [28]:
df.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 359 entries, 2022-03-04 09:15:00+05:30 to 2022-03-10 14:02:46+05:30
Data columns (total 10 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   Open             359 non-null    float64
 1   High             359 non-null    float64
 2   Low              359 non-null    float64
 3   Close            359 non-null    float64
 4   Adj Close        359 non-null    float64
 5   Volume           359 non-null    int64  
 6   Supertrend       359 non-null    bool   
 7   Final Lowerband  202 non-null    float64
 8   Final Upperband  148 non-null    float64
 9   rsi              359 non-null    float64
dtypes: bool(1), float64(8), int64(1)
memory usage: 36.5 KB


In [32]:
dates_dict = defaultdict(list)

dates_dict['Date'].append('1-2-3')

In [33]:
dates_dict

defaultdict(list, {'Date': ['1-2-2', '1-2-3']})